## Content Based Recommendation based on clothing characteristics and Weather Data Points

Preconditions for the recommendation system to work is for the clothing dataset to be built acording to the personalized dataset maker guidelines.

#### IMPORTS

In [74]:
import requests
import json
import random
import pandas as pd # Import pandas for easier data handling
from pathlib import Path
from datetime import datetime

#### CONFIGURATION PARAMETERS

In [75]:
# --- CONFIGURATION ---
key = open("openweatherkey.txt", "r")
WEATHER_API_KEY = key.read()
key.close()

OUTPUT_FILE = Path("dataset/personalized_clothing_dataset_female.json") 

# --- Load Content from JSON ---
try:
    with open(OUTPUT_FILE, 'r') as f:
        # Load the JSON data into a Python list of dictionaries
        wardrobe_data_list = json.load(f)

    # Convert the list into a Pandas DataFrame for easy filtering and access
    df_wardrobe = pd.DataFrame(wardrobe_data_list)
    
    print(f"Successfully loaded {len(df_wardrobe)} records from the dataset.")
    print("\n--- Sample Data (DataFrame Head) ---")
    print(df_wardrobe.head())

except FileNotFoundError:
    print(f"Error: The file {OUTPUT_FILE} was not found. Please ensure it is in the correct directory.")
except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from {OUTPUT_FILE}. The file may be empty or corrupted.")

# --- OUTPUT RECOMMENDATION CONFIGURATION ---
TARGET_CITY = "Seoul"
OUTPUT_FILE_NAME = "recommended_outfit.txt"

# --- WARDROBE HISTORY FOR DIVERSIFICATION LAYER ---
HISTORY_FILE = Path("wardrobe_history.json")
def load_history():
    """Loads the history log from a JSON file."""
    if HISTORY_FILE.exists():
        with open(HISTORY_FILE, 'r') as f:
            try:
                return json.load(f)
            except json.JSONDecodeError:
                return {}
    return {}

# Load history once at the start
WARDROBE_HISTORY = load_history()


Successfully loaded 20 records from the dataset.

--- Sample Data (DataFrame Head) ---
                                          image_link  category  \
0  simulated_wardrobes/Female_Wardrobe/Tops_04_Ka...   t-shirt   
1  simulated_wardrobes/Female_Wardrobe/Outers_03_...      coat   
2  simulated_wardrobes/Female_Wardrobe/Bottoms_08...     jeans   
3  simulated_wardrobes/Female_Wardrobe/Bottoms_07...     skirt   
4  simulated_wardrobes/Female_Wardrobe/Bottoms_04...  trousers   

      outer_inner                                              shape  \
0           inner  {'sleeve': 'short-sleeve', 'neckline': 'crew-n...   
1           outer  {'sleeve': 'long-sleeve', 'neckline': 'collar'...   
2           inner  {'sleeve': 'none', 'neckline': 'none', 'fit': ...   
3  not-applicable  {'sleeve': 'none', 'neckline': 'none', 'fit': ...   
4           inner  {'sleeve': 'none', 'neckline': 'none', 'fit': ...   

                      material            color              pattern  \
0          

#### WEATHER FETCHING

Fetch the current weather to then be used in layer 1 of the system

In [76]:
## --- WEATHER FETCHING ---
def get_weather(city):
    """Fetches weather data from OpenWeatherMap and extracts key values."""
    if not WEATHER_API_KEY:
        print("ERROR: Please set your OpenWeatherMap API key.")
        return None
        
    url = (
        f"https://api.openweathermap.org/data/2.5/weather"
        f"?q={city}&appid={WEATHER_API_KEY}&units=metric"
    )

    try:
        response = requests.get(url)
        response.raise_for_status() # Raise exception for HTTP errors (4xx or 5xx)
        data = response.json()
        
        temp = data["main"]["temp"]
        wind = data["wind"]["speed"]
        rain_1h = data.get("rain", {}).get("1h", 0) # mm in last 1 hour
        
        weather_desc = data["weather"][0]["description"]
        
        print(f"Weather in {city}: {temp:.1f}°C, {weather_desc}")
        
        return {
            "city": city,
            "temp": temp,
            "wind": wind,
            "rain": rain_1h,
            "desc": weather_desc
        }
        
    except requests.exceptions.HTTPError as e:
        print(f"ERROR fetching weather for {city}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


### RECOMMENDATION SYSTEM BUILDING

#### MAPPING OUT GARMENT TYPES AND CATEGORIES TO FACILITATE OUTFIT CREATION

In [77]:
GARMENT_TYPES = {
    "Outer": ["outer"],
    "Top": ["inner"],
    "Bottom": ["inner", "not-applicable"] # Bottoms (jeans, shorts) and dresses are marked as 'not-applicable' in layering, so we must include both.
}
# Map garment category names to required component
CATEGORY_MAP = {
    "jacket": "Outer", "coat": "Outer", "hoodie": "Outer",
    "t-shirt": "Top", "button-up shirt": "Top", "sweater": "Top",
    "jeans": "Bottom", "trousers": "Bottom", "shorts": "Bottom", "skirt": "Bottom",
    "dress": "Dress"
}

#### RECSYS LAYER 1 : Content-Based Filtering
##### Ranking garments by category and weather appropriatness
Set threshold for clothing scores that we will look for based on that weather/environment.
The environment parameters are simplified as warmth needs that can be influenced by wind, impermeability and layering needs.

In [78]:
## --- ENVIRONMENTAL SCORING ---
def compute_required_scores(weather_data):
    """
    Translates temperature, wind, and rain into required garment scores (1-5 for warmth/layering, 1-3 for impermeability).
    """
    temp = weather_data["temp"]
    rain = weather_data["rain"]
    wind = weather_data["wind"]

    # Warmth Requirement (Higher score = warmer clothes needed)
    # Scale: Cold (1) to Hot (5) is reversed to required warmth (5) to (1).
    if temp < 5:      warmth_req = 5 # Very cold
    elif temp < 15:   warmth_req = 4 # Cool
    elif temp < 25:   warmth_req = 3 # Mild
    elif temp < 32:   warmth_req = 2 # Warm
    else:             warmth_req = 1 # Hot

    # Adjust for wind chill (simplified)
    if wind > 8: warmth_req += 1
    warmth_req = min(5, warmth_req)
    
    # Impermeability Requirement (1=none, 3=high)
    if rain >= 2.5: # Heavy rain
        impermeability_req = 3
    elif rain > 0.5: # Light rain/drizzle
        impermeability_req = 2
    else:
        impermeability_req = 1
        
    # Layering Base (The system encourages layering if temperature is volatile or cold)
    # We'll use 4 for default layering to promote flexibility unless it's hot.
    layering_req = 4 if warmth_req >= 3 else 3

    return {
        "warmth": warmth_req,
        "impermeability": impermeability_req,
        "layering": layering_req
    }

#### RECSYS LAYER 2: PREFERENCE IMPACT

In [79]:
# TODO : Implement recsys based on preferred styles

#### RECSYS LAYER 3: DIVERSIFICATION FACTOR

Diversification based recommendation requires for the system to remember its last recommended outfit.

Two features are required : 
-   History log saving of recently recommended outfits details
-   Diversification penalty calculator for garments with different cooldown time before recommending based on category


In [80]:
def save_outfit_to_history(recommended_outfit):
    """
    Updates the wardrobe history log with the items selected for the current outfit.
    """
    if not isinstance(recommended_outfit, dict) or recommended_outfit.get('Outfit Type') not in ['Layered', 'Dress']:
        # Only save valid, successful outfit dictionaries
        return

    # Load existing history or initialize a new one
    current_history = load_history()
    now_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    items_to_log = []

    # Handle Layered Outfit
    if recommended_outfit['Outfit Type'] == 'Layered':
        if recommended_outfit['Outerwear']:
            items_to_log.append(recommended_outfit['Outerwear'])
        items_to_log.append(recommended_outfit['Top'])
        items_to_log.append(recommended_outfit['Bottom'])

    # Handle Dress Outfit
    elif recommended_outfit['Outfit Type'] == 'Dress':
        items_to_log.append(recommended_outfit['Dress'])

    # Update History Dictionary
    for item in items_to_log:
        image_link = item.get('image_link')
        if image_link:
            # Log the image link (as the unique item ID) and the current timestamp
            current_history[image_link] = now_str
            
    # Save the Updated History
    with open(HISTORY_FILE, 'w') as f:
        json.dump(current_history, f, indent=4)
    
    # Update the global variable for use in the current session (if needed)
    global WARDROBE_HISTORY
    WARDROBE_HISTORY = current_history 
    print(f"History updated with {len(items_to_log)} items.")

In [81]:
from datetime import datetime, timedelta

# Define the specific cooldown rules
DIVERSITY_RULES = {
    'Top': {'cooldown_hours': 48, 'penalty': -7.0},
    'Bottom': {'cooldown_hours': 72, 'penalty': -3.0},
    'Outer': {'cooldown_hours': 0, 'penalty': 0.0}, # No functional cooldown for Outerwear
    'Dress': {'cooldown_hours': 24, 'penalty': 0.0}
}

# Updated function signature to accept component_type_str directly
def calculate_diversity_penalty(item, component_type_str):
    """Calculates the tiered penalty based on garment type and usage history."""
    
    # Use the passed string directly (Fixed the TypeError: unhashable type: 'dict')
    rule = DIVERSITY_RULES.get(component_type_str, DIVERSITY_RULES['Outer']) 
    
    # Outerwear/Dresses often have a 0 penalty, so check that first
    if rule['penalty'] == 0.0 or rule['cooldown_hours'] == 0:
        return 0.0 

    # --- Apply Cooldown Penalty for Tops/Bottoms ---
    image_link = item.get('image_link')
    last_worn_str = WARDROBE_HISTORY.get(image_link)
    
    if last_worn_str:
        try:
            last_worn_time = datetime.strptime(last_worn_str, '%Y-%m-%d %H:%M:%S')
            time_difference = datetime.now() - last_worn_time
            
            # Check if the item was worn within its specific cooldown period
            if time_difference < timedelta(hours=rule['cooldown_hours']):
                return rule['penalty'] 
        except ValueError:
            pass

    return 0.0 # No penalty

#### RECSYS LAYER 4: COLOR CLASHING PENALTY

In [82]:
def get_color_clash_penalty(item_A, item_B):
    """
    Assigns a penalty if the colors of two items clash aesthetically.
    (This is a simple placeholder rule for demonstration.)
    Returns a penalty value (e.g., 0 for good match, -5 for bad clash).
    """
    color_A = item_A.get('color', '').lower()
    color_B = item_B.get('color', '').lower()
    
    # Define a simple list of clashing primary colors for a harsh penalty
    clash_pairs = [
        ('red', 'green'), ('blue', 'orange'), ('purple', 'yellow'),
        ('red', 'bright blue'), ('pink', 'dark blue')
    ]
    
    # Check for direct clash or highly saturated clashes
    for c1, c2 in clash_pairs:
        if (c1 in color_A and c2 in color_B) or (c1 in color_B and c2 in color_A):
            return -5.0 # Severe penalty for clash

    # Simple rule: If both items are "bright" or have strong patterns, apply moderate penalty
    if ('bright' in color_A or 'graphic' in item_A.get('pattern', '')) and \
       ('bright' in color_B or 'graphic' in item_B.get('pattern', '')):
        return -2.0 # Moderate penalty for being too busy

    # Reward simple, neutral combinations
    neutral_colors = ['black', 'white', 'grey', 'dark blue', 'navy', 'beige']
    is_A_neutral = any(n in color_A for n in neutral_colors)
    is_B_neutral = any(n in color_B for n in neutral_colors)
    
    if is_A_neutral and is_B_neutral:
        return 1.0 # Small bonus for safe choice (not a penalty, but a reward)

    return 0.0 # No penalty/reward

#### Final recommendation results

Greedy strategy to build the outfit using a diversification index and making color penalty to decide what to recommend

In [83]:
def rank_garments(garments_df, required_scores, component_type):
    """
    Ranks garments based on how well their attributes meet or exceed the required scores,
    with corrected filtering logic.
    """
    df = garments_df.copy()

    # --- SETUP (Assume CATEGORY_MAP and GARMENT_TYPES are defined globally or passed in) ---
    
    # --- Filter by Garment Category (Primary Filter) ---
    if component_type == "Dress":
        # Special case: only look for 'dress' category
        df_filtered = df[df['category'] == 'dress'].copy()
    else:
        # General case: Filter by category (T-shirt, Jeans, Jacket, etc.)
        valid_categories = [cat for cat, comp in CATEGORY_MAP.items() if comp == component_type]
        if not valid_categories:
            return [] # No category map found

        df_filtered = df[df['category'].isin(valid_categories)].copy()
        
        # Secondary filter: Check inner/outer layer classification
        df_filtered = df_filtered[df_filtered['outer_inner'].isin(GARMENT_TYPES[component_type])].copy()

    if df_filtered.empty:
        return []
        
    # --- Calculate Content-Based Matching ---
    # Warmth: Minimize the difference between item warmth and required warmth.
    df_filtered['warmth_fit'] = 10 - abs(df_filtered['warmth_score'] - required_scores['warmth'])
    # Impermeability: Must meet or exceed the requirement. Penalty if too low.
    impermeability_penalty = 5 
    df_filtered['impermeability_fit'] = df_filtered['impermeability_score'].apply(
        lambda x: 10 if x >= required_scores['impermeability'] else 10 - impermeability_penalty
    )
    # Layering: Favor items with high layering score, especially if required layering is high.
    df_filtered['layering_fit'] = df_filtered['layering_score']
    

    # --- Calculate Diversity Penalty ---
    df_filtered['diversity_penalty'] = df_filtered.apply(
        # Passing the row dict and the component_type string explicitly
        lambda row: calculate_diversity_penalty(row.to_dict(), component_type), axis=1
    )
    # Debug print added:
    print(f"--- Diversity Penalty for {component_type} ---")
    print(df_filtered[['image_link', 'diversity_penalty']].head(3))
    
    # --- Calculate Final Total Score (Weighted Sum) ---
    # Combine Weather, Preference, and Diversity Penalty.
    
    df_filtered['total_score'] = (
        (df_filtered['warmth_fit'] * 0.40) + 
        (df_filtered['impermeability_fit'] * 0.25) + 
        (df_filtered['layering_fit'] * 0.15) + 
        # (df_filtered['preference_score'] * 10 * 0.20) 
        # Subtract the penalty score
        + df_filtered['diversity_penalty'] 
    )

    # --- Return Results ---
    # Return the top 3 best-fitting items as a list of dictionaries
    return df_filtered.sort_values(by='total_score', ascending=False).head(3).to_dict('records')


In [84]:
def recommend_outfit(wardrobe_df, city):
    """Finds the best combination of Outer, Top, and Bottom for the weather."""
    
    weather_data = get_weather(city)
    if not weather_data:
        return "Failed to get weather data."
        
    required_scores = compute_required_scores(weather_data)
    print("\nRequired Scores:", required_scores)
    
    # --- Check for a Dress first ---
    dress_ranking = rank_garments(wardrobe_df, required_scores, "Dress")
    if dress_ranking and len(dress_ranking) > 0 and required_scores['warmth'] <= 3:
        # CRITICAL FIX: Access the first item in the list (index 0)
        best_dress_item = dress_ranking[0] 
        
        # If a dress is available and it's not too cold (Warmth Req <= 3), recommend it
        return {
            "Outfit Type": "Dress",
            "Weather": weather_data,
            "Required Scores": required_scores,
            "Dress": best_dress_item # Use the corrected item reference
        }

    # --- Assemble Layered Outfit (Outer + Top + Bottom) ---
    
    # Get ranked items for each component
    top_ranks = rank_garments(wardrobe_df, required_scores, "Top")
    bottom_ranks = rank_garments(wardrobe_df, required_scores, "Bottom")
    outer_ranks = rank_garments(wardrobe_df, required_scores, "Outer")
    
    if not top_ranks or not bottom_ranks:
        return "ERROR: Missing essential items (Top or Bottom) in the wardrobe."

    # Select the best combination (Greedy Strategy: Outer -> Top -> Bottom)
    
    # Greedy Choice for Outerwear (Priority 1)
    best_outer = None
    if required_scores['warmth'] >= 3 and outer_ranks:
        # Select the highest weather-scoring Outerwear first
        best_outer = outer_ranks[0]
    
    # Greedy Choice for Top (Priority 2: Dependent on Outerwear)
    top_scores_modified = []
    dominating_item = best_outer if best_outer else None 

    for top_item in top_ranks:
        clash_penalty = 0
        if dominating_item:
            clash_penalty = get_color_clash_penalty(dominating_item, top_item)
        
        # Note: top_item['total_score'] already includes Layer 3 (Diversity Penalty)
        modified_score = top_item['total_score'] + clash_penalty
        top_scores_modified.append({'item': top_item, 'modified_score': modified_score})

    # Select the Top with the highest MODIFIED score
    best_top_selection = sorted(
        top_scores_modified, 
        key=lambda x: x['modified_score'], 
        reverse=True
    )
    best_top = best_top_selection[0]['item']
    
    
    # Greedy Choice for Bottom (Priority 3: Dependent on Top)
    bottom_scores_modified = []
    
    for bottom_item in bottom_ranks:
        clash_penalty = get_color_clash_penalty(best_top, bottom_item)
        
        # Note: bottom_item['total_score'] already includes Layer 3 (Diversity Penalty)
        modified_score = bottom_item['total_score'] + clash_penalty
        bottom_scores_modified.append({'item': bottom_item, 'modified_score': modified_score})

    # Select the Bottom with the highest MODIFIED score
    best_bottom_selection = sorted(
        bottom_scores_modified, 
        key=lambda x: x['modified_score'], 
        reverse=True
    )
    best_bottom = best_bottom_selection[0]['item']
    
    
    # --- Final Output ---
    outfit = {
        "Outfit Type": "Layered",
        "Weather": weather_data,
        "Required Scores": required_scores,
        "Outerwear": best_outer,
        "Top": best_top,
        "Bottom": best_bottom,
    }
    
    return outfit

In [85]:
# --- RUN RECOMMENDATION ---
recommended_outfit = recommend_outfit(df_wardrobe, TARGET_CITY)

# Save the result to the history log
if isinstance(recommended_outfit, dict):
    save_outfit_to_history(recommended_outfit)

Weather in Seoul: 7.8°C, broken clouds

Required Scores: {'warmth': 4, 'impermeability': 1, 'layering': 4}
--- Diversity Penalty for Dress ---
                                           image_link  diversity_penalty
9   simulated_wardrobes/Female_Wardrobe/Dresses_03...                0.0
19  simulated_wardrobes/Female_Wardrobe/Dresses_02...                0.0
--- Diversity Penalty for Top ---
                                           image_link  diversity_penalty
0   simulated_wardrobes/Female_Wardrobe/Tops_04_Ka...               -7.0
7   simulated_wardrobes/Female_Wardrobe/Tops_05_Ka...                0.0
11  simulated_wardrobes/Female_Wardrobe/Tops_07_Ka...               -7.0
--- Diversity Penalty for Bottom ---
                                          image_link  diversity_penalty
2  simulated_wardrobes/Female_Wardrobe/Bottoms_08...               -3.0
3  simulated_wardrobes/Female_Wardrobe/Bottoms_07...                0.0
4  simulated_wardrobes/Female_Wardrobe/Bottoms_04...       

In [86]:
# Open the file for writing ('w')
with open(OUTPUT_FILE_NAME, 'w') as f:
    
    # --- Write Header ---
    f.write(f"--- OUTFIT RECOMMENDATION for {TARGET_CITY} ({datetime.now().strftime('%Y-%m-%d %H:%M:%S')}) ---\n\n")

    # --- Write Detailed JSON Result ---
    f.write("--- DETAILED JSON OUTPUT ---\n")
    if isinstance(recommended_outfit, dict):
        # Write the JSON string to the file
        f.write(json.dumps(recommended_outfit, indent=4))
        f.write("\n\n")

        # --- Write Readable Summary ---
        f.write("--- READABLE SUMMARY ---\n")
        
        if recommended_outfit['Outfit Type'] == 'Layered':
            f.write(f"Outfit Type: Layered\n")
            
            # Write Outerwear
            outerwear_note = "None recommended (weather is warm)."
            outerwear_warmth = "N/A"
            if recommended_outfit["Outerwear"]:
                outerwear_note = recommended_outfit['Outerwear']['notes']
                outerwear_warmth = recommended_outfit['Outerwear']['warmth_score']
            f.write(f"Outerwear: {outerwear_note} (Warmth: {outerwear_warmth})\n")
            
            # Write Top
            top_note = recommended_outfit['Top']['notes']
            top_warmth = recommended_outfit['Top']['warmth_score']
            f.write(f"Top:       {top_note} (Warmth: {top_warmth})\n")
            
            # Write Bottom
            bottom_note = recommended_outfit['Bottom']['notes']
            bottom_warmth = recommended_outfit['Bottom']['warmth_score']
            f.write(f"Bottom:    {bottom_note} (Warmth: {bottom_warmth})\n")
        
        elif recommended_outfit['Outfit Type'] == 'Dress':
            # Write Dress
            dress_note = recommended_outfit['Dress']['notes']
            dress_warmth = recommended_outfit['Dress']['warmth_score']
            f.write(f"Dress:     {dress_note} (Warmth: {dress_warmth})\n")
            
    else:
        # Handle the error case (if recommend_outfit returned a string error message)
        f.write(f"Recommendation Failed: {recommended_outfit}\n")
        f.write("Please verify your API key and city name.")

print(f"Recommendation saved successfully to {OUTPUT_FILE_NAME}")

Recommendation saved successfully to recommended_outfit.txt
